In [ ]:
import os
from dotenv import load_dotenv
from typing import TypedDict, Annotated

load_dotenv()


True

In [ ]:
from langchain_groq import ChatGroq
from langchain_deepseek import ChatDeepSeek
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.checkpoint.memory import MemorySaver


In [ ]:
LLMS = {}

LLMS['gemini'] = ChatGoogleGenerativeAI(
    api_key = os.getenv("GOOGLE_API_KEY"),
    model = 'gemini-1.5-flash'
)

LLMS['deepseek'] = ChatDeepSeek(
    api_key=os.getenv("OPENROUTER_API_KEY"),
    api_base=os.getenv("OPENROUTER_BASE_URL"),
    model="deepseek/deepseek-chat-v3-0324:free"
)

LLMS['moonshotai'] = ChatGroq(
    api_key = os.getenv("GROQ_API_KEY"),
    model = "moonshotai/kimi-k2-instruct"
)


In [ ]:
llm = LLMS['gemini']


In [ ]:
class ChatState(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]


In [ ]:
def chat_node(state: ChatState):
    messages = state['messages'];
    response = llm.invoke(messages);
    state['messages'] = [response]
    
    return state


In [ ]:
graph = StateGraph(ChatState)

graph.add_node('chat_node', chat_node)

graph.add_edge(START, "chat_node")
graph.add_edge("chat_node", END)

checkpointer = MemorySaver()

chatbot = graph.compile(checkpointer=checkpointer)


In [ ]:
thread_id = 1;
config = {
    "configurable": {
        "thread_id": thread_id
    }
}

while True:
    
    human_message = input(">>>");
    
    if human_message.strip().lower() in ['quit', 'exit', 'bye']:
        print("User Pressed", human_message)
        break;
    
    print("User Message:", human_message)
    
    initial_state = {
        'messages': [HumanMessage(content=human_message)]
    }
    
    state = chatbot.invoke(initial_state, config=config)
    
    print("AI Message:", state['messages'][-1].content)
